In [1]:
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
driver = webdriver.Edge()

In [13]:
driver.get('https://ajaxinate.myshopify.com/')

In [16]:
# Pagina con scroll infinito
last_height = driver.execute_script("return document.body.scrollHeight")
last_height

1721

In [17]:
ajax_elements = driver.find_elements(By.XPATH, "//a[contains(@href, 'page=')]")
ajax_elements

[<selenium.webdriver.remote.webelement.WebElement (session="57ed5e8e109be63803410244c12419c3", element="f.40777410015E2B977EFDAECCF64D5859.d.BD725C03A1FC4331D46EA14943BFB418.e.133")>]

In [19]:
# Lista para almacenar las URLs de las llamadas AJAX
ajax_calls = [] # 1

# Espera explícita
wait = WebDriverWait(driver, 10) # 2

# Mientras se pueda hacer scroll, continúa haciéndolo y recoge las URLs
try:
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True: # 3
        # Realiza scroll hasta el final de la página
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 4

        # Espera a que se cargue la página
        time.sleep(3)  # Ajustar si es necesario

        # Guarda la llamada AJAX si es que se encuentra
        new_height = driver.execute_script("return document.body.scrollHeight") # 5
        if new_height == last_height:
            break
        last_height = new_height

        # Encuentra todos los elementos que podrían ser la llamada AJAX
        # Aquí se asume que la URL de la llamada AJAX tiene un patrón identificable
        ajax_elements = driver.find_elements(By.XPATH, "//a[contains(@href, 'page=')]") # 6
        for element in ajax_elements:
            ajax_call = element.get_attribute('href')
            if ajax_call not in ajax_calls:
                ajax_calls.append(ajax_call)

except Exception as e: # 7
    print("Ocurrió un error:", e)
#finally:
    # Cierra el navegador
    #driver.quit()

# Imprime las llamadas AJAX capturadas
for call in ajax_calls: # 8
    print(call)

https://ajaxinate.myshopify.com/?page=3
https://ajaxinate.myshopify.com/?page=4


In [20]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

In [21]:
api_url = 'https://salcobrand.cl/api/v2/products/store_stock?state_id=375&sku=571707'

In [22]:
response = requests.get(api_url, headers=headers)
response.text

'[{"id":9,"code":10,"name":"Avda. Apoquindo N° 5855 - 5879, Las Condes, Santiago","lat":"-33.4088948","lng":"-70.56916","schedules":{"normal_schedule":"Lunes a Viernes: 08:30-21:00","weekend_schedules":["Sábado: 09:00-21:00","Domingo: 10:00-20:00"],"special_schedules":[]},"stocks":{"571707":7},"availabilities":{"571707":true}},{"id":16,"code":21,"name":"Avda. Tomas Moro N° 20, Las Condes, Santiago","lat":"-33.4083914","lng":"-70.55182359999999","schedules":{"normal_schedule":"Lunes a Viernes: 08:30-21:00","weekend_schedules":["Sábado: 09:00-21:00","Domingo: 10:00-21:00"],"special_schedules":[]},"stocks":{"571707":8},"availabilities":{"571707":true}},{"id":37,"code":50,"name":"Avda. Las Condes N° 12145, Las Condes, Santiago","lat":"-33.3746293","lng":"-70.5218307","schedules":{"normal_schedule":"Lunes a Domingo: 00:01-23:59","weekend_schedules":[],"special_schedules":[]},"stocks":{"571707":39},"availabilities":{"571707":true}},{"id":52,"code":73,"name":"Avda. Las Condes N° 8231, Las Con

In [23]:
json_data = response.json()
json_data

[{'id': 9,
  'code': 10,
  'name': 'Avda. Apoquindo N° 5855 - 5879, Las Condes, Santiago',
  'lat': '-33.4088948',
  'lng': '-70.56916',
  'schedules': {'normal_schedule': 'Lunes a Viernes: 08:30-21:00',
   'weekend_schedules': ['Sábado: 09:00-21:00', 'Domingo: 10:00-20:00'],
   'special_schedules': []},
  'stocks': {'571707': 7},
  'availabilities': {'571707': True}},
 {'id': 16,
  'code': 21,
  'name': 'Avda. Tomas Moro N° 20, Las Condes, Santiago',
  'lat': '-33.4083914',
  'lng': '-70.55182359999999',
  'schedules': {'normal_schedule': 'Lunes a Viernes: 08:30-21:00',
   'weekend_schedules': ['Sábado: 09:00-21:00', 'Domingo: 10:00-21:00'],
   'special_schedules': []},
  'stocks': {'571707': 8},
  'availabilities': {'571707': True}},
 {'id': 37,
  'code': 50,
  'name': 'Avda. Las Condes N° 12145, Las Condes, Santiago',
  'lat': '-33.3746293',
  'lng': '-70.5218307',
  'schedules': {'normal_schedule': 'Lunes a Domingo: 00:01-23:59',
   'weekend_schedules': [],
   'special_schedules': 

In [24]:
rows = []

for store in json_data:
    store_id = store.get('id')
    code = store.get('code')
    name = store.get('name')
    lat = store.get('lat')
    lng = store.get('lng')
    normal_schedule = store['schedules'].get('normal_schedule')
    weekend_schedules = '; '.join(store['schedules'].get('weekend_schedules', []))
    special_schedules = '; '.join(store['schedules'].get('special_schedules', []))

    for product_id, stock in store['stocks'].items():
        availability = store['availabilities'].get(product_id, False)
        rows.append({
            'store_id': store_id,
            'code': code,
            'name': name,
            'latitude': lat,
            'longitude': lng,
            'normal_schedule': normal_schedule,
            'weekend_schedules': weekend_schedules,
            'special_schedules': special_schedules,
            'product_id': product_id,
            'stock': stock,
            'availability': availability
        })

# Crear el DataFrame
df = pd.DataFrame(rows)

# Mostrar el DataFrame
df

,store_id,code,name,latitude,longitude,normal_schedule,weekend_schedules,special_schedules,product_id,stock,availability
0,9,10,"Avda. Apoquindo N° 5855 - 5879, Las Condes, Sa...",-33.4088948,-70.56916,Lunes a Viernes: 08:30-21:00,Sábado: 09:00-21:00; Domingo: 10:00-20:00,,571707,7,True
1,16,21,"Avda. Tomas Moro N° 20, Las Condes, Santiago",-33.4083914,-70.55182359999999,Lunes a Viernes: 08:30-21:00,Sábado: 09:00-21:00; Domingo: 10:00-21:00,,571707,8,True
2,37,50,"Avda. Las Condes N° 12145, Las Condes, Santiago",-33.3746293,-70.5218307,Lunes a Domingo: 00:01-23:59,,,571707,39,True
3,52,73,"Avda. Las Condes N° 8231, Las Condes, Santiago",-33.3983044,-70.551147,Lunes a Viernes: 09:00-21:00,Sábado y Domingo: 10:00-21:00,,571707,8,True
4,57,81,"Avda. Cuarto Centenario N° 933, Las Condes, Sa...",-33.417108,-70.5544287,Lunes a Viernes: 09:30-20:30,Sábado: 10:00-16:00; Domingo: Cerrado,,571707,4,True
5,60,85,"Avda. Colón N° 6064, Las Condes, Santiago",-33.4231818,-70.5311935,Lunes a Viernes: 08:00-22:00,Sábado y Domingo: 09:00-22:00,,571707,8,True
6,69,100,"Avda. Padre Hurtado N° 40, Las Condes, Santiago",-33.3941372,-70.5458286,Lunes a Domingo: 00:01-23:59,,,571707,10,True
7,71,103,"Avda. Francisco Bilbao N°8750 Local 3048, Supe...",-33.4287187,-70.5405628,Lunes a Sábado: 08:30-20:30,Domingo: 09:00-20:30,,571707,3,True
8,75,108,"Avda. Américo Vespucio N° 1463, Mall / Strip C...",-33.4006416,-70.58692289999999,Lunes a Sábado: 09:00-21:00,Domingo: 10:00-21:00,,571707,9,True
9,79,115,"Avda. Apoquindo N° 4671, Las Condes, Santiago",-33.4131426,-70.5807516,Lunes a Viernes: 09:00-21:00,Sábado: 10:00-17:00; Domingo: Cerrado,,571707,2,True
